In [ ]:
## 使用 ResNet 来尝试训练解决 CIFAR10  的分类问题。

In [1]:
import torch
from torch.utils.data import random_split
import torchvision
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision.transforms as transforms
import torch.nn.functional as F
from PIL import Image

In [2]:
# 数据处理

transform_train = torchvision.transforms.Compose(
    [
        torchvision.transforms.RandomCrop(32, padding=2),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.RandomGrayscale(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.25, 0.25, 0.25])
    ]
)

transform_test = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.5, 0.5, 0.5], [0.25, 0.25, 0.25])
    ]
)

train_set = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform_train) #训练数据集
print(type(train_set))
print(len(train_set))
# train_set, val_set = random_split(train_set, [45000, 5000]) # 将训练集划分为训练集和验证集
# print(type(train_set))
# print(type(val_set))
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform_test)

# train_set = [[Image.fromarray(image), target] for image, target in train_set]
# val_set = [[Image.fromarray(image), target] for image, target in val_set]

train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True, num_workers=4)   #生成一个个batch进行批训练，组成batch的时候顺序打乱取
# val_loader = torch.utils.data.DataLoader(val_set, batch_size=1000, shuffle = False, num_workers=4) 
test_loader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False, num_workers=4)

<class 'torchvision.datasets.cifar.CIFAR10'>
50000


##### ResNet 实现
参考：
1. https://blog.csdn.net/winycg/article/details/86709991
2. https://blog.csdn.net/sunqiande88/article/details/80100891

In [3]:
![jupyter](./20180426215052446.png)

In [ ]:
# 用于ResNet18和34的残差块，用的是2个3x3的卷积
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inChannelCount, outChannelCount, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(inChannelCount, outChannelCount, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(outChannelCount)
        self.conv2 = nn.Conv2d(outChannelCount, outChannelCount, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(outChannelCount)
        self.shortcut = nn.Sequential()
        # 经过处理后的x要与x的维度相同(尺寸和深度)
        # 如果不相同，需要添加卷积+BN来变换为同一维度
        if stride != 1 or inChannelCount != self.expansion*outChannelCount:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inChannelCount, self.expansion*outChannelCount,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*outChannelCount)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.inChannelCount = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        
        self.layer1 = self._sequential(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._sequential(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._sequential(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._sequential(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _sequential(self, block, outChannelCount, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.inChannelCount, outChannelCount, stride))
            self.inChannelCount = outChannelCount * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def createResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

In [4]:
net = createResNet18()
print(net)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [7]:
# 从学习率 = 0.01开始训练1个epoch， loss： 1.402，val_loss: 1.499
optimizer = torch.optim.Adam(net.parameters(), lr=0.001) 

lossFunc = torch.nn.CrossEntropyLoss()

net.train()

losses = []
losses_val = []
for epoch in range(1):
    for i, data in enumerate(train_loader, 0):
        x_batch, y_batch = data
        predictData = net(x_batch)
            
        loss = lossFunc(predictData, y_batch)
        losses.append(loss.data.numpy())
        
        if i % 10 == 0:
            print(epoch, i, "----loss", loss.data.numpy())
            val_data = [data for i, data in enumerate(test_loader, 0) if i == 0][0]
            y_val = net(val_data[0])
            loss_val = lossFunc(y_val, val_data[1])
            print("----val loss", loss_val.data.numpy())
            losses_val.append(loss_val.data.numpy())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

0 0 ----loss 1.4291242
----val loss 1.4997045
0 10 ----loss 1.3706359
----val loss 1.369659
0 20 ----loss 1.4619173
----val loss 1.3378009
0 30 ----loss 1.3787068
----val loss 1.3228652
0 40 ----loss 1.2302798
----val loss 1.3309627
0 50 ----loss 1.2665075
----val loss 1.3160684
0 60 ----loss 1.2292455
----val loss 1.3547552
0 70 ----loss 1.3158573
----val loss 1.2963223
0 80 ----loss 1.3623327
----val loss 1.2849746
0 90 ----loss 1.266409
----val loss 1.2763331
0 100 ----loss 1.3504081
----val loss 1.2935337
0 110 ----loss 1.5139548
----val loss 1.2861918
0 120 ----loss 1.2713747
----val loss 1.2604018
0 130 ----loss 1.2545682
----val loss 1.2443454
0 140 ----loss 1.2344018
----val loss 1.214889
0 150 ----loss 1.3083956
----val loss 1.2475243
0 160 ----loss 1.4304171
----val loss 1.2704363
0 170 ----loss 1.3596852
----val loss 1.1601884
0 180 ----loss 1.1267571
----val loss 1.2447544
0 190 ----loss 1.3172306
----val loss 1.176367
0 200 ----loss 1.3930887
----val loss 1.1591133
0 210 -

In [9]:
torch.save(net, "Touch_ResNot_Model2")

d:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:360: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
d:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:360: UserWarning: Couldn't retrieve source code for container of type BasicBlock. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [4]:
net = torch.load("Touch_ResNot_Model2")
net.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [7]:
from collections import Counter
import numpy

    

In [16]:
test_loader2 = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False, num_workers=4)
# 取一次数据
test_datas = [data for i, data in enumerate(test_loader2, 0)]
y_test = []
y_predict = []
for test_data in test_datas:
    x_test_temp = test_data[0]
    y_test_temp = test_data[1]
    y_predict_temp = net(x_test_temp)
    y_test += list(y_test_temp.data.numpy())
    y_predict += list(y_predict_temp.data.numpy())
    print('-')
    break

-


In [17]:

def see_result(y, y_predict):
    Y_Result = []
    for i in range(y_predict.shape[0]):
        Y_Result.append( numpy.argmax(y_predict[i]) )
    Y_Result = numpy.array(Y_Result)
    counter = Counter(list((Y_Result == y.reshape(-1))))
    print(counter)
    print("accuracy:",  counter[True]/len(y))
    yTypeCountor = Counter(y.reshape(-1))
    print("y 类型对应的数量：", yTypeCountor)
    print()
    
see_result(numpy.array(y_test), numpy.array(y_predict))
# see_result(y_train, net(x_train))

Counter({True: 113, False: 15})
accuracy: 0.8828125
y 类型对应的数量： Counter({6: 19, 8: 17, 3: 15, 0: 13, 7: 13, 9: 11, 2: 11, 1: 10, 4: 10, 5: 9})



CPU 训练太慢！尝试用GPU训练

In [10]:
isGPU = "cuda" if torch.cuda.is_available() else "cpu"
print("isGPU:",isGPU)
device = torch.device(isGPU)

isGPU: cpu


In [ ]:
net = net.to(device)

optimizer = torch.optim.Adam(net.parameters(), lr=0.001) 

lossFunc = torch.nn.CrossEntropyLoss()

net.train()

losses = []
losses_val = []
for epoch in range(1):
    for i, data in enumerate(train_loader, 0):
        x_batch, y_batch = data
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        predictData = net(x_batch)
            
        loss = lossFunc(predictData, y_batch)
        losses.append(loss.data.numpy())
        
        if i % 10 == 0:
            print(epoch, i, "----loss", loss.data.numpy())
            val_data = [data for i, data in enumerate(test_loader, 0) if i == 0][0]
            y_val = net(val_data[0].to(device))
            loss_val = lossFunc(y_val, val_data[1])
            print("----val loss", loss_val.data.numpy())
            losses_val.append(loss_val.data.numpy())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()











